# Feature Generation

In [3]:
# Importing necessary libraries
import gs.dmarc
import numpy as np
import pandas as pd
from autocorrect import Speller
from nltk.corpus import wordnet as wn
from pathlib import Path

/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdo

### Read file

In [4]:
#reading the .csv file and declare datatype to avoid low memory issues
df = pd.read_csv("preprocessed.csv",dtype='unicode')

## Add new features to the dataset

###  check if the subject contains special characters

In [5]:
def if_special(x):
    special_characters = "]!@#$%^&*()-+?_=,<>/["
    for c in x :
        if c in special_characters:
            return 1
    return 0

In [6]:
df['special_characters_exists_subject'] = df['Subject'].apply(if_special)

### count number of words in subject

In [7]:
def count_num_words(x):
    w = x.split(" ")
    return len(w)

In [8]:
df['number_of_words_subject'] = df['Subject'].apply(count_num_words)

### calculate total time

from hopper import Hopper


### count number of capitalized words in subject

In [9]:
def count_num_cap_words(x):
    w = x.split(" ")
    count = 0
    for i in w:
        if i.isupper():
            count+=1
    return count

In [10]:
df['number_of_capitalized_words_subject'] = df['Subject'].apply(count_num_cap_words)

### count number of capitalized characters in subject

In [11]:
def count_num_cap_char(x):
    count = 0
    for i in x:
        if i.isupper():
            count+=1
    return count

In [12]:
df['number_of_capitalized_characters_subject'] = df['Subject'].apply(count_num_cap_char)

### count number of digits in subject

In [13]:
def count_digit(x):
    count=0
    for i in x:
        if i.isdigit():
            count+=1
    return count

In [14]:
df['number_of_digits_subject'] = df['Subject'].apply(count_digit)

### count number of characters in subject

In [15]:
def count_num_char(x):
    count=0
    for i in x:
        if i.isalpha():
            count+=1
    return count

In [16]:
df['number_of_characters_subject'] = df['Subject'].apply(count_num_char)

###  Total number of spaces in Subject 

In [17]:
def count_space(x):
    count=0
    for i in x:
        if i.isspace():
            count+=1
    return count

In [18]:
df['number_of_spaces_subject'] = df['Subject'].apply(count_space)

### Total number of special characters in Subject 

In [19]:
def count_special(x):
    special_characters = "]!@#$%^&*()-+?_=,<>/["
    return len([c for c in x if c in special_characters])

In [20]:
df['number_of_special_characters_subject'] = df['Subject'].apply(count_special)

### Number of single quotes in Subject 

In [21]:
def singleQuote(x):
    count = 0
    for res in x:
        if "'" in res:
            count+=1
    save = count/2
    return save

In [22]:
df['number_of_single_Quotes_subject'] = df['Subject'].apply(singleQuote)

### Number of semi-colons in Subject 

In [23]:
def count_num_semiColon(x):
    count = 0
    for i in x:
        if ';' in i:
            count+=1
    return count


In [24]:
df['number_of_semiColon_subject'] = df['Subject'].apply(count_num_semiColon)

### Ratio upper case  / lower case in subject

In [25]:
def ratio_upperCase_lowerCae(x):

    countUpp =0
    countLow =0

    save = x.split(" ")
    for i in save:
        if i.isupper():
            countUpp+=1
        else:
            countLow+=1

    ratio = countUpp/countLow

    return ratio

In [26]:
df['ratio_of_uppercase/lowercase_words'] = df['Subject'].apply(ratio_upperCase_lowerCae)

### Total number of uppercase words 

In [27]:
def upperCase(x):
    count = 0
    save = x.split(" ")
    for i in save:
        if i.isupper():
            count+=1
    return count


In [28]:
df['Total_number_of_upperCase'] = df['Subject'].apply(upperCase)

### Max. Word length (number of characters on the longest word in Subject) 

In [29]:
def MaxWordLength(str): 
    strLen = len(str) 
    save = 0; currentLength = 0
      
    for i in range(0, strLen): 
        if (str[i] != ' '): 
            currentLength += 1
        else: 
            save = max(save, currentLength) 
            currentLength = 0

    return max(save, currentLength) 

In [30]:
df['Max_word_length_in_subject'] = df['Subject'].apply(MaxWordLength)

### Is External Domain (0 or 1)

In [ ]:
def isExternalDomain(from_address, to_address):
    print(from_address)
    from_domain = from_address.split('@')
    to_domain = to_address.split('@')
    
    if from_domain == to_domain:
        return 0
    else:
        return 1

In [ ]:
df['is_external_domain'] = df.apply(df.apply(lambda x: isExternalDomain(x.From, x.To), axis=1))

### Check for Spelling Mistakes (0 or 1)

In [ ]:
def containsSpellingMistakes(str):
    words = str.split(' ')
    spell = Speller(lang='en')
    
    for word in words:
        if spell(word) != word:
            return 1
    
    return 0

In [ ]:
df['contains_spelling_mistakes'] = df['Subject'].apply(containsSpellingMistakes)

### Action Verbs in Subject (0 or 1)

In [ ]:
def containsActionVerbsSubject(str):
    words = str.split(' ')
    
    for word in words:
        if wn.synsets(word)[0].pos() == 'v':
            return 1
        
    return 0

In [ ]:
df['contains_action_verbs_subject'] = df['Subject'].apply(containsActionVerbsSubject)

### Dmarc Protected (0 or 1)

In [ ]:
def isDmarcProtected(str):
    host = str[1].split('@')[1]
    policy = gs.dmarc.receiver_policy(str)
    
    if (policy in (gs.dmarc.ReceiverPolicy.quarantine, gs.dmarc.ReceiverPolicy.reject)):
        return 0
    else:
        return 1

In [ ]:
df['is_dmarc_protected'] = df['From'].apply(isDmarcProtected)

### Number of Receivers

In [ ]:
def numberOfReceivers(str):
    num_emails = str.count('<')
    if num_emails == 0:
        return 1
    else:
        return num_emails

In [ ]:
df['number_of_receivers'] = df['To'].apply(numberOfReceivers)

### Is Mail User Agent Empty (0 or 1)

In [ ]:
def isMailUserAgentEmpty(str):
    if str == 'unknown':
        return 1
    return 0

In [ ]:
df['is_mail_user_agent_empty'] = df['X-Mailer'].apply(isMailUserAgentEmpty)

In [ ]:
df

## Length of the Subject

In [ ]:
def length_subject():
    df['Subject_length']  = df['Subject'].str.len()

## Date Validation

In [ ]:
def date_validation:
    df['Date'] = df['Date'].str[:-2]
    #validating date after converting it to datetime
    df['new_date'] = pd.to_datetime(df['Date'],errors="coerce")
    df['validate_date'] = np.where(df['new_date']< datetime.now(), True, False)

## Black listed/White Listed

In [ ]:
def validate_blackListed():
    df['new_email'] = df['From'].str.extract(r'([\w\.-]+@[\w\.-]+)')
    df['domain'] = df['new_email'].apply(str).str.split('@').str[1]
    list_urls = df['domain'].astype(str).values.tolist()
    black_listed = []
    white_listed = []
    for i in list_urls:
        try:
            if(i == 'nan'):
                continue
            elif(i in SPAMHAUS_DBL ):
                print(i)
                black_listed.append(i)
            else:
                print(i)
                white_listed.append(i)
        except:
            pass
    for index,row in df.iterrows():
        if(df['domain'].iloc[index] in black_listed ):
            df.iloc[index, df.columns.get_loc('blackListed')] = True
        else:
            df.iloc[index, df.columns.get_loc('blackListed')] = False


### Split dataset to train and test

In [30]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.3)
train.to_csv('train_data.csv',index=False)
test.to_csv('test_data.csv',index=False)